# Deep Q learning with Doom 🕹️
In this notebook we'll implement an agent <b>that plays Doom by using a Deep Q learning architecture.</b> <br>
Our agent playing Doom:

<img src="assets/doom.gif" style="max-width: 600px;" alt="Deep Q learning with Doom"/>


## This notebook is part of the Free Deep Reinforcement Course 📝
<img src="https://simoninithomas.github.io/Deep_reinforcement_learning_Course/assets/img/preview.jpg" alt="Deep Reinforcement Course" style="width: 500px;"/>

<p> Deep Reinforcement Learning Course is a free series of blog posts about Deep Reinforcement Learning, where we'll learn the main algorithms, <b>and how to implement them in Tensorflow.</b></p>

<p>The goal of these articles is to <b>explain step by step from the big picture</b> and the mathematical details behind it, to the implementation with Tensorflow </p>


<a href="https://simoninithomas.github.io/Deep_reinforcement_learning_Course/">Syllabus</a><br>
<a href="https://medium.freecodecamp.org/an-introduction-to-reinforcement-learning-4339519de419">Part 0: Introduction to Reinforcement Learning </a><br>
<a href=""> Part 1: Q-learning with FrozenLake</a><br>
<a href=""> Part 2: Deep Q-learning with Doom</a><br>
<a href=""> Part 3: Policy Gradients with Doom </a><br>


## Any questions 👨‍💻
<p> If you have any questions, feel free to ask me: </p>
<p> 📧: <a href="mailto:hello@simoninithomas.com">hello@simoninithomas.com</a>  </p>
<p> Github: https://github.com/simoninithomas/Deep_reinforcement_learning_Course </p>
<p> 🌐 : https://simoninithomas.github.io/Deep_reinforcement_learning_Course/ </p>
<p> Twitter: <a href="https://twitter.com/ThomasSimonini">@ThomasSimonini</a> </p>
<p> Don't forget to <b> follow me on <a href="https://twitter.com/ThomasSimonini">twitter</a>, <a href="https://github.com/simoninithomas/Deep_reinforcement_learning_Course">github</a> and <a href="https://medium.com/@thomassimonini">Medium</a> to be alerted of the new articles that I publish </b></p>
    
## Important note 🤔
<b> You can run it on your computer but it's better to run it on GPU based services </b> (except if your computer have GPUs or you're able to wait 10 years 😅), personally I use Microsoft Azure and their Deep Learning Virtual Machine (they offer 170$)
https://azuremarketplace.microsoft.com/en-us/marketplace/apps/microsoft-ads.dsvm-deep-learning
<br>
⚠️ I don't have any business relations with them. I just loved their excellent customer service.

If you have some troubles to use Microsoft Azure follow the explainations of this excellent article here (without last the part fast.ai): https://medium.com/@manikantayadunanda/setting-up-deeplearning-machine-and-fast-ai-on-azure-a22eb6bd6429

## Step 1: Importing the libraries 📚

In [ ]:
import tensorflow as tf      # Deep Learning library
import numpy as np           # Handle matrices
from vizdoom import *        # Doom Environment
import random                # Handling random number generation
import time                  # Handling time calculation
from skimage import transform# Help us to preprocess the frames

from collections import deque# Ordered collection with ends
import matplotlib.pyplot as plt # Display graphs

## Step 2: Create our environment 🎮
- Now that we imported the libraries/dependencies, we will create our environment.
- Doom environment takes:
    - A configuration files that handle all the options (size of the frame, possible actions...)
    - A scenario files: that generates the correct scenario (in our case basic but you're invited to try other scenarios).
- Note: We transform actions to an identity array but instead of using 0 and 1 we use a bool identity. This will help us to do one_hot encoding.
 

In [ ]:
"""
Here we create our environment
"""
def create_environment():
    game = DoomGame()
    
    # Load the correct configuration
    game.load_config("basic.cfg")
    
    # Load the correct scenario (in our case basic scenario)
    game.set_doom_scenario_path("basic.wad")
    
    game.init()
    
    # Here our possible actions
    left = [1, 0, 0]
    right = [0, 1, 0]
    shoot = [0, 0, 1]
    possible_actions = [left, right, shoot]
    
    return game, possible_actions
       
"""
Here we performing random action to test the environment
"""
def test_environment():
    game = DoomGame()
    game.load_config("basic.cfg")
    game.set_doom_scenario_path("basic.wad")
    game.init()
    shoot = [0, 0, 1]
    left = [1, 0, 0]
    right = [0, 1, 0]
    actions = [shoot, left, right]

    episodes = 10
    for i in range(episodes):
        game.new_episode()
        while not game.is_episode_finished():
            state = game.get_state()
            img = state.screen_buffer
            misc = state.game_variables
            action = random.choice(actions)
            print(action)
            reward = game.make_action(action)
            #print ("\treward:", reward)
            time.sleep(0.02)
        print ("Result:", game.get_total_reward())
        time.sleep(2)

In [ ]:
game, possible_actions = create_environment()

In [ ]:
# Training hyperparameters
total_episodes = 5000
max_steps = 100

# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob


# Q-learning settings
learning_rate =  0.0002
gamma = 0.99


batch_size = 64

frame_repeat = 4

pretrain_length = batch_size
memory_size = 50000

state_size = [84,84,4]
action_size = 3

## Step 3 : Define the preprocessing functions ⚙️
### preprocess_frame
Preprocessing is an important step, <b>because we want to reduce the complexity of our states to reduce the computation time needed for training.</b>
<br><br>
Our steps:
- Grayscale each of our frames (because <b> color does not add important information </b>). But this is already done by the config file.
- Crop the screen (in our case we remove the roof because it contains no information)
- We normalize pixel values
- Finally we resize the preprocessed frame

In [ ]:
"""
    preprocess_frame:
    Take a frame.
    Resize it.
        __________________
        |                 |
        |                 |
        |                 |
        |                 |
        |_________________|
        
        to
        _____________
        |            |
        |            |
        |            |
        |____________|
    Normalize it.
    
    return preprocessed_frame
    
    """
def preprocess_frame(frame):
    # Greyscale frame already done in our vizdoom config
    # x = np.mean(frame,-1)
    
    # Crop the screen (remove the roof because it contains no information)
    cropped_frame = frame[30:-10,30:-30]
    
    # Normalize Pixel Values
    normalized_frame = cropped_frame/255.0
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, [84,84])
    
    return preprocessed_frame

### stack_frames
👏 This part was made possible thanks to help of <a href="https://github.com/Miffyli">Anssi</a><br>
Stacking frames is really important because it helps us to give have a sense of motion to our Neural Network.
- First we preprocess frame
- Then we append the frame to the deque that automatically removes the oldest frame
- Finally we build the stacked state

This is how work stack:
- For the first frame, we feed the other 3 with blank frames
- At each timestep, we add the new frame to deque and then we stack them to form a new stacked frame
- And so on
<img src="assets\stack.png" alt="stack">

In [ ]:
stack_size = 4
# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4) 

def stack_frames(stacked_frames, state):
    # Preprocess frame
    frame = preprocess_frame(state)
        
    # Append frame to deque, automatically removes the oldest frame
    stacked_frames.append(frame)
       
    # Build the stacked state (first dimension specifies different frames)
    stacked_state = np.stack(stacked_frames, axis=2)
    
    return stacked_state

## Step 3: Create our Deep Q learning Neural Network model 🧠

In [ ]:
class DQNetwork:
    def __init__(self, state_size, action_size, learning_rate, name='DQNetwork'):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        
        with tf.variable_scope(name):
            # We create the placeholders
            self.inputs_ = tf.placeholder(tf.float32, [None, 84, 84, 4], name="inputs")
            self.actions_ = tf.placeholder(tf.float32, [None, 3], name="actions_")
            self.target_Q = tf.placeholder(tf.float32, [None], name="target")
            
            """
            First convnet:
            CNN
            BatchNormalization
            ELU
            """
            # Input is 84x84x4
            self.conv1 = tf.layers.conv2d(inputs = self.inputs_,
                                         filters = 32,
                                         kernel_size = [8,8],
                                         strides = [4,4],
                                         padding = "VALID",
                                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                         name = "conv1")
             
            self.conv1_batchnorm = tf.layers.batch_normalization(self.conv1,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                     name = 'batch_norm1')
            
            self.conv1_out = tf.nn.elu(self.conv1_batchnorm, name="conv1_out")
            
            """
            Second convnet:
            CNN
            BatchNormalization
            ELU
            """
            self.conv2 = tf.layers.conv2d(inputs = self.conv1_out,
                                 filters = 64,
                                 kernel_size = [4,4],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv2")
        
            self.conv2_batchnorm = tf.layers.batch_normalization(self.conv2,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                     name = 'batch_norm2')

            self.conv2_out = tf.nn.elu(self.conv2_batchnorm, name="conv2_out")
            
            """
            Third convnet:
            CNN
            BatchNormalization
            ELU
            """
            self.conv3 = tf.layers.conv2d(inputs = self.conv2_out,
                                 filters = 128,
                                 kernel_size = [4,4],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv3")
        
            self.conv3_batchnorm = tf.layers.batch_normalization(self.conv3,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                     name = 'batch_norm3')

            self.conv3_out = tf.nn.elu(self.conv3_batchnorm, name="conv3_out")

            self.flatten = tf.layers.flatten(self.conv3_out)
            
            self.fc = tf.layers.dense(inputs = self.flatten,
                                  units = 512,
                                  activation = tf.nn.elu,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                name="fc1")
            
            self.output = tf.layers.dense(inputs = self.fc, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                          units = 3, 
                                        activation=None)


  
            ### Train with loss (targetQ - Q)^2
            # output has length 2, for two actions. This next line chooses
            # one value from output (per row) according to the one-hot encoded actions.
            self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_), axis=1)
            
            
            # (Qtarget - Qhat)
            self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))
            
            self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)

In [ ]:
tf.reset_default_graph()
DQNetwork = DQNetwork(state_size, action_size, learning_rate)

## Step: Experience Replay
This part was made possible by the excellent notebook: _ by udacity

In [ ]:
class Memory():
    def __init__(self, max_size):
        self.buffer = deque(maxlen = max_size)
    
    def add(self, experience):
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        buffer_size = len(self.buffer)
        index = np.random.choice(np.arange(buffer_size),
                                size = batch_size,
                                replace = False)
        
        return [self.buffer[i] for i in index]

<p> Here we'll deal with the empty memory problem: we pre-populate our memory by taking random actions and storing the experience (state, action, reward, new_state).</p>

In [ ]:
# Render the environment
game.new_episode()

# Instantiate memory
memory = Memory(max_size = memory_size)

for i in range(pretrain_length):
    
    if i == 0:
        # First we need a state
        state = game.get_state().screen_buffer
        state = stack_frames(stacked_frames, state)
    
    # Random action
    action = random.choice(possible_actions)
    
    # Get the rewards
    reward = game.make_action(action)
    
    # Look if the episode is finished
    done = game.is_episode_finished()
    
    
    if done:
        # We finished the episode
        next_state = np.zeros(state.shape)
        
        # Add experience to memory
        memory.add((state, action, reward, next_state, done))
        # Start a new episode
        game.new_episode()
    else:
        # Get the next state
        next_state = game.get_state().screen_buffer
        next_state = stack_frames(stacked_frames, next_state)
        # Add experience to memory
        memory.add((state, action, reward, next_state, done))
        state = next_state

## Step 4: Train our Model 🏃‍♂️

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode $\leftarrow 1$ **to** $M$ **do**
  * Observe $s_0$
  * **For** $t \leftarrow 0$ **to** $T-1$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s_t,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

In [ ]:
# Setup TensorBoard Writer
writer = tf.summary.FileWriter("/tmp/dqn/new1")
tf.summary.scalar('Loss', DQNetwork.loss)
write_op = tf.summary.merge_all()

In [ ]:
rewards_list = []

with tf.Session() as sess:
    # Initialize the variables
    sess.run(tf.global_variables_initializer())
    
    # Init the game
    game.init()
    
    decay_step = 0
    
    for episode in range(total_episodes):
        # Make new episode
        game.new_episode()
        step = 0
        
        # Observe the first state
        frame = game.get_state().screen_buffer
        state = stack_frames(stacked_frames, frame)
        
        while step < max_steps:
            decay_step +=1
            
            ## EPSILON GREEDY STRATEGY
            # Choose action a from state s using epsilon greedy.
            ## First we randomize a number
            exp_exp_tradeoff = np.random.rand()
            
            # Here we'll use an improved version of our epsilon greedy strategy used in Q-learning notebook
            explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
            
            
            if (explore_probability > exp_exp_tradeoff):
                # Make a random action
                action = random.choice(possible_actions)
                
            else:
                # Get action from Q-network
                # Estimate the Qs values state
                Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state.reshape((1, *state.shape))})

                # Take the biggest Q value (= the best action)
                action = np.argmax(Qs)

                action = possible_actions[int(action)]
           
        
            # Do the action
            reward = game.make_action(action)
            
            # Look if the episode is finished
            done = game.is_episode_finished()
            
            
            # If the game is finished
            if done:
                # the episode ends so no next state
                next_state = np.zeros((84,84), dtype=np.int)
                next_state = stack_frames(stacked_frames, next_state)
            
                step = max_steps
                                      
                print('Episode: {}'.format(episode),
                      'Total reward: {}'.format(game.get_total_reward()),
                      'Training loss: {:.4f}'.format(loss),
                      'Explore P: {:.4f}'.format(explore_probability))
                rewards_list.append((episode, game.get_total_reward()))
                
                memory.add((state, action, reward, next_state, done))
                 
            else:
                 # Get the next state
                next_state = game.get_state().screen_buffer
                next_state = stack_frames(stacked_frames, next_state)
                # Add experience to memory
                memory.add((state, action, reward, next_state, done))
                state = next_state
                step += 1
            
            ### LEARNING PART            
            # Obtain random mini-batch from memory
            batch = memory.sample(batch_size)
            states = np.array([each[0] for each in batch], ndmin=3)
            actions = np.array([each[1] for each in batch])
            rewards = np.array([each[2] for each in batch]) 
            next_states = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            
            target_Qs_batch = []
            
            # Get Q values for next_state 
            target_Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: next_states})
            
            for i in range(0, len(batch)):
                terminal = dones[i]
                
                # If we are in a terminal state, only equals reward
                if terminal:
                    target_Qs_batch.append(rewards[i])
                else:
                    target = rewards[i] + gamma * np.max(target_Qs[i])
                    target_Qs_batch.append(target)
                
            
            targets = np.array([each for each in target_Qs_batch])
    
            loss, _ = sess.run([DQNetwork.loss, DQNetwork.optimizer],
                                feed_dict={DQNetwork.inputs_: states,
                                           DQNetwork.target_Q: targets,
                                           DQNetwork.actions_: actions})
        
            
        
            # Write TF Summaries
            summary = sess.run(write_op, feed_dict={DQNetwork.inputs_: states,
                                           DQNetwork.target_Q: targets,
                                           DQNetwork.actions_: actions})
            writer.add_summary(summary, episode)
            writer.flush()
                    


## Step 5: Watch our Agent play 👀

In [ ]:
for i in range(10):
    game.new_episode()
    while not game.is_episode_finished():
        frame = game.get_state().screen_buffer
        state = stack_frames(stacked_frames, frame)
        # Take the biggest Q value (= the best action)
        Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state.reshape((1, *state.shape))})
        action = np.argmax(Qs)
        action = possible_actions[int(action)]
        game.make_action(action)        
        score = game.get_total_reward()
print("Total score: ", score)
